In [1]:
import numpy as np
import cupy as cp
from scipy.io import loadmat
import os
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import shutil
import h5py
from scipy.signal import lfilter

%matplotlib inline

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
sys.path.append('../github/pykilosort')
sys.path.append('../github/phylib')
from pykilosort import Bunch, run, add_default_handler
from pykilosort.utils import memmap_large_array, LargeArrayWriter

In [3]:
dat_path = Path('D:\kilosort-testing\\100s_data\\p1\\imec_385_100s.bin')
dir_path = dat_path.parent
add_default_handler(level='INFO')

In [4]:
rez_path = Path('D:\kilosort-testing\\100s_data\\matlab_stages')
test_path = Path('D:\kilosort-testing\\100s_data\\python_stages')
dataset_name = 'imec_385_100s'

In [5]:
def get_rez(rez_loc):
    rez_file = h5py.File(rez_loc)
    main_key = list(rez_file.keys())[-1]
    rez = Bunch()
    for key in rez_file[main_key].keys():
        try:
            rez[key] = rez_file[main_key][key][()].squeeze()
        except AttributeError:
            pass
    for key in rez_file[main_key]['ops'].keys():
        try:
            rez[key] = rez_file[main_key]['ops'][key][()].squeeze()
        except AttributeError:
            pass
    return rez

def get_ctx(ctx_loc):
    files = os.listdir(ctx_loc)
    ctx = Bunch()
    for file in files:
        if file[-3:] == 'npy':
            ctx[file[:-4]] = np.load(ctx_loc / file)
    return ctx

def transpose_fortran(array):
    return np.asfortranarray(array.T)

def _save(array, path, name):
    np.save(path / name, transpose_fortran(array))
    
def _save_largearray(array, path, name):
    writer = LargeArrayWriter(path / (name + '.dat'), dtype = np.float32, shape = (*array.shape[:-1], -1))
    writer.append(np.asfortranarray(array))
    writer.close()

def setup_dir(path, name):
    test_path = path / name / '.kilosort' / dataset_name
    if os.path.isdir(test_path):
        shutil.rmtree(test_path)
    os.makedirs(test_path)
    return test_path

def test(name, rez, ctx, mapping=None, mapping_axes=None, python_name=None, atol=1e-08):
    
    if python_name is None:
        python_name = name
    var_m = np.copy(rez[name]).T
    var_p = cp.asnumpy(ctx.intermediate[python_name])
    
    if mapping_axes is not None:
        assert mapping is not None
        for i in mapping_axes:
            var_m = np.take(var_m, mapping, axis=i)
    
    return np.allclose(var_m, var_p, atol=atol)

def test_abs(name, rez, ctx, mapping=None, mapping_axes=None, python_name=None, atol=1e-08):
    
    if python_name is None:
        python_name = name
    var_m = np.abs(np.copy(rez[name]).T)
    var_p = np.abs(cp.asnumpy(ctx.intermediate[python_name]))
    
    if mapping_axes is not None:
        assert mapping is not None
        for i in mapping_axes:
            var_m = np.take(var_m, mapping, axis=i)
    
    return np.allclose(var_m, var_p, atol=atol)

## Test Preprocess

In [6]:
test_pre_path = setup_dir(test_path, 'test_preprocess')

In [7]:
probe = Bunch()
probe.NchanTOT = 385
# WARNING: indexing mismatch with MATLAB hence the -1
probe.chanMap = np.load(dir_path / 'chanMap.npy').squeeze().astype(np.int64) - 1
probe.xc = np.load(dir_path / 'xc.npy').squeeze()
probe.yc = np.load(dir_path / 'yc.npy').squeeze()
probe.kcoords = np.load(dir_path / 'kcoords.npy').squeeze()

In [10]:
ctx = run(dat_path, probe=probe, dir_path=test_pre_path.parent.parent, n_channels=385, dtype=np.int16, sample_rate=3e4, stop_after='preprocess')

22:01:35.620 [I] main:56              Loaded raw data with 385 channels, 3000000 samples.
22:01:35.622 [I] utils:334            Starting step good_channels.
Finding good channels: 100%|███████████████████████████████████████████████████████████| 46/46 [00:29<00:00,  1.57it/s]
22:02:05.014 [I] preprocess:312       Found 83221 threshold crossings in 100.00 seconds of data.
22:02:05.015 [I] preprocess:313       Found 81/374 bad channels.
22:02:05.015 [I] utils:344            Step `good_channels` took 29.39s.
22:02:05.015 [I] utils:334            Starting step whitening_matrix.
Computing the whitening matrix: 100%|████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.50it/s]
22:02:11.342 [I] preprocess:231       Computed the whitening matrix.
22:02:11.353 [I] utils:344            Step `whitening_matrix` took 6.34s.
22:02:11.379 [I] utils:334            Starting step preprocess.
22:02:11.380 [I] preprocess:349       Loading raw data and applying filters.
Preprocessing: 

In [11]:
rez_pre = get_rez(rez_path / 'rez_preprocess.mat')

In [12]:
rez_pre.keys()

dict_keys(['Wrot', 'xc', 'xcoords', 'yc', 'ycoords', 'AUCsplit', 'GPU', 'NT', 'NTbuff', 'Nbatch', 'Nchan', 'NchanTOT', 'Nfilt', 'Th', 'ThPre', 'chanMap', 'fbinary', 'fproc', 'fs', 'fshigh', 'igood', 'kcoords', 'lam', 'minFR', 'minfr_goodchannels', 'momentum', 'nPCs', 'nSkipCov', 'nfilt_factor', 'nskip', 'nt0', 'nt0min', 'ntbuff', 'reorder', 'sampsToRead', 'scaleproc', 'sigmaMask', 'spkTh', 'tend', 'trange', 'tstart', 'twind', 'useRAM', 'whiteningRange'])

In [19]:
ir = ctx.intermediate
ir.keys()

dict_keys(['Nbatch', 'igood', 'Wrot', 'proc_path'])

In [22]:
np.allclose(ir.igood, rez_pre.igood)

True

In [25]:
rez_pre.Wrot[:5,:5]

array([[61.9022727 , -3.39832827, -4.9992118 , -4.02359851, -4.84960377],
       [-3.39830592, 61.04385853, -3.92501876, -4.23253328, -3.43119279],
       [-4.99919727, -3.9250236 , 71.22970819, -4.25783359, -5.21818139],
       [-4.02358659, -4.23250869, -4.25782129, 77.90967822, -4.6552442 ],
       [-4.84959893, -3.4311939 , -5.21820039, -4.65524793, 78.61017585]])

In [26]:
ir.Wrot[:5,:5]

array([[62.07098108, -3.42985521, -5.01939981, -4.04189645, -4.87181909],
       [-3.42985521, 61.18618388, -3.93719626, -4.26883095, -3.4435032 ],
       [-5.01939981, -3.93719626, 71.39750067, -4.26265096, -5.23968566],
       [-4.04189645, -4.26883095, -4.26265096, 78.06118443, -4.67152417],
       [-4.87181909, -3.4435032 , -5.23968566, -4.67152417, 78.76671274]])

## Test Batch Reordering

In [6]:
test_cluster_path = setup_dir(test_path, 'test_cluster')

rez_preprocess = get_rez(rez_path / 'rez_preprocess.mat')

In [7]:
variables = ['igood', 'Wrot']

for variable in variables:
    _save(rez_preprocess[variable], test_cluster_path, variable)
    
shutil.copyfile(rez_path / 'temp_wh.dat', test_cluster_path / 'proc.dat')

WindowsPath('D:/kilosort-testing/100s_data/python_stages/test_cluster/.kilosort/imec_385_100s/proc.dat')

In [8]:
del rez_preprocess

In [9]:
probe = Bunch()
probe.NchanTOT = 385
# WARNING: indexing mismatch with MATLAB hence the -1
probe.chanMap = np.load(dir_path / 'chanMap.npy').squeeze().astype(np.int64) - 1
probe.xc = np.load(dir_path / 'xc.npy').squeeze()
probe.yc = np.load(dir_path / 'yc.npy').squeeze()
probe.kcoords = np.load(dir_path / 'kcoords.npy').squeeze()

In [11]:
ctx = run(dat_path, probe=probe, dir_path=test_cluster_path.parent.parent, n_channels=385, dtype=np.int16, sample_rate=3e4, stop_after='preprocess')

17:40:37.508 [I] main:56              Loaded raw data with 385 channels, 3000000 samples.


## Test Learn

## Test Merge

In [7]:
test_merge_path = setup_dir(test_path, 'test_merge')

rez_learn = get_rez(rez_path / 'rez_learn.mat')

In [8]:
def save_rez_learn(rez, path):
    
    variables = ['ccbsort', 'dWU', 'igood', 'iNeigh', 'iNeighPC', 'iorig', 'mu', 
                 'simScore', 'U', 'U_a', 'U_b', 'UA', 'W', 'W_a', 'W_b',
                'WA', 'Wrot', 'wPCA', 'wTEMP']
    
    for variable in variables:
        _save(rez[variable], path, variable)
        
    _save(rez.ccb, path, 'ccb0')
    
    _save_largearray(rez.cProj, path, 'fW')
    _save_largearray(rez.cProjPC, path, 'fWPC')
    
    st3 = np.copy(rez.st3)
    st3[1,:] = rez.st3[1,:] - 1 # 0-index channels
    _save(st3, path, 'st3')
    
    shutil.copyfile(path / 'fW.dat', path / 'proc.dat') # hack to avoid re-running the pre-processing stage

In [9]:
save_rez_learn(rez_learn, test_merge_path)
del rez_learn

In [10]:
probe = Bunch()
probe.NchanTOT = 385
# WARNING: indexing mismatch with MATLAB hence the -1
probe.chanMap = np.load(dir_path / 'chanMap.npy').squeeze().astype(np.int64) - 1
probe.xc = np.load(dir_path / 'xc.npy').squeeze()
probe.yc = np.load(dir_path / 'yc.npy').squeeze()
probe.kcoords = np.load(dir_path / 'kcoords.npy').squeeze()

In [11]:
ctx = run(dat_path, probe=probe, dir_path=test_merge_path.parent.parent, n_channels=385, dtype=np.int16, sample_rate=3e4, stop_after='merge')

23:25:29.215 [I] main:56              Loaded raw data with 385 channels, 3000000 samples.
23:25:29.356 [I] utils:334            Starting step merge.
Finding merges:   0%|                                                                          | 0/357 [00:00<?, ?it/s]../github/pykilosort\pykilosort\postprocess.py:426: RuntimeWarning: invalid value encountered in true_divide
  Q = (Qi / max(Q00, Q01)).min()
Finding merges: 100%|███████████████████████████████████████████████████████████████| 357/357 [00:03<00:00, 114.03it/s]
23:25:32.765 [I] utils:344            Step `merge` took 3.41s.


In [12]:
rez_merge = get_rez(rez_path / 'rez_merge.mat')

For the merge phase it is enough to check that st3 is the same

In [13]:
st3_matlab = np.copy(rez_merge.st3.T)
st3_matlab[:,1] = st3_matlab[:,1] - 1
st3_python = cp.asnumpy(ctx.intermediate.st3_m)
np.allclose(st3_matlab, st3_python)

True

In [14]:
ix = np.where(st3_matlab[:,1] - st3_python[:,1] != 0)[0]
bad_clusters = np.unique(np.concatenate((st3_python[:,1][ix], st3_matlab[:,1][ix])))
print(f'The following clusters were different: {bad_clusters}')

The following clusters were different: []


In [15]:
del ctx
del rez_merge

## Test First Split

In [16]:
test_split1_path = setup_dir(test_path, 'test_split1')
rez_merge = get_rez(rez_path / 'rez_merge.mat')

In [17]:
def save_rez_merge(rez, path):
   
    variables = ['ccbsort', 'dWU', 'igood', 'iNeigh', 'iNeighPC', 'iorig', 'mu', 
                 'simScore', 'U', 'U_a', 'U_b', 'UA', 'W', 'W_a', 'W_b',
                'WA', 'Wrot', 'wPCA', 'wTEMP']
    
    for variable in variables:
        _save(rez[variable], path, variable)
        
    _save(rez.ccb, path, 'ccb0')
    
    _save_largearray(rez.cProj, path, 'fW')
    _save_largearray(rez.cProjPC, path, 'fWPC')
    
    st3 = np.copy(rez.st3)
    st3[1,:] = rez.st3[1,:] - 1 # 0-index channels
    _save(st3, path, 'st3')
    _save(st3, path, 'st3_m')
    
    shutil.copyfile(path / 'fW.dat', path / 'proc.dat')

In [18]:
save_rez_merge(rez_merge, test_split1_path)
del rez_merge

In [19]:
probe = Bunch()
probe.NchanTOT = 385
# WARNING: indexing mismatch with MATLAB hence the -1
probe.chanMap = np.load(dir_path / 'chanMap.npy').squeeze().astype(np.int64) - 1
probe.xc = np.load(dir_path / 'xc.npy').squeeze()
probe.yc = np.load(dir_path / 'yc.npy').squeeze()
probe.kcoords = np.load(dir_path / 'kcoords.npy').squeeze()

In [20]:
ctx = run(dat_path, probe=probe, dir_path=test_split1_path.parent.parent, n_channels=385, dtype=np.int16, sample_rate=3e4, \
          stop_after='split_1')

23:25:36.662 [I] main:56              Loaded raw data with 385 channels, 3000000 samples.
23:25:36.804 [I] utils:334            Starting step split_1.
23:25:37.532 [I] postprocess:522      Found 0 splits, checked 0/357 clusters, nccg 0
23:25:42.824 [I] postprocess:522      Found 4 splits, checked 100/361 clusters, nccg 8
23:25:47.425 [I] postprocess:522      Found 10 splits, checked 200/367 clusters, nccg 9
23:25:52.510 [I] postprocess:522      Found 13 splits, checked 300/370 clusters, nccg 14
23:25:58.552 [I] postprocess:732      Finished splitting. Found 19 splits, checked 376/376 clusters, nccg 19
23:25:58.741 [I] utils:344            Step `split_1` took 21.94s.


In [21]:
rez_split1 = get_rez(rez_path / 'rez_split1.mat')

In [22]:
st3_matlab = np.copy(rez_split1.st3.T)
st3_matlab[:,1] = st3_matlab[:,1] - 1
st3_python = cp.asnumpy(ctx.intermediate.st3_s1)

In [23]:
ix = np.where(st3_matlab[:,1] - st3_python[:,1] != 0)[0]
bad_clusters = np.unique(np.concatenate((st3_python[:,1][ix], st3_matlab[:,1][ix])))
print(f'The following clusters were different: {bad_clusters}')

The following clusters were different: [ 44. 173. 195. 246. 357. 358. 359. 360. 361. 362. 363. 365. 369. 371.
 372. 373. 374. 375.]


Some of these differences are just due to labelling, we can try and learn a mapping from the Python clusters to their corresponding Matlab clusters:

In [24]:
cluster_mapping = np.zeros((len(bad_clusters), 2), dtype = int)
cluster_mapping[:,0] = bad_clusters
cluster_mapping[:,1] = -1

for i in range(len(bad_clusters)):
    ix_p = np.where(st3_python[:,1] == bad_clusters[i])[0]
    for j in bad_clusters:
        ix_m = np.where(st3_matlab[:,1] == j)[0]
        if np.array_equal(ix_p, ix_m):
            cluster_mapping[i,1] = j
            break

mapping_found = False
if np.sum(cluster_mapping[:,1] == -1):
    print(f"No mapping found as the following Python clusters can't be matched:" \
        f"{cluster_mapping[:,0][np.where(cluster_mapping[:,1] == 0)[0]]}")
elif np.max(st3_matlab[:,1]) != np.max(st3_python[:,1]):
    print("Some Matlab clusters had no corresponding Python match")
else:
    mapping = np.arange(np.max(st3_python[:,1] + 1), dtype=int)
    for i in range(cluster_mapping.shape[0]):
        mapping[cluster_mapping[i,0]] = cluster_mapping[i,1]
    mapping_found = True
    print("Mapping was found")

Mapping was found


In [25]:
print(f"mu matches: {test('mu', rez_split1, ctx, mapping, [0], 'mu_s')}")
print(f"simScore matches: {test('simScore', rez_split1, ctx, mapping, [0,1], 'simScore_s')}")
print(f"isplit matches: {test('isplit', rez_split1, ctx, mapping, [0,1])}")

print(f"iNeighPC matches: {np.allclose(cp.asnumpy(ctx.intermediate.iNeighPC_s.T), (rez_split1.iNeighPC-1)[mapping])}")

mu matches: True
simScore matches: True
isplit matches: True
iNeighPC matches: True


SVD components only match up to an arbitrary sign so we compare absolute values and also use a higher tolerance

In [26]:
print(f"W matches: {test_abs('W', rez_split1, ctx, mapping, [1], 'W_s', atol=1e-05)}")
print(f"U matches: {test_abs('U', rez_split1, ctx, mapping, [1], 'U_s', atol=1e-05)}")
print(f"Wphy matches: {test_abs('Wphy', rez_split1, ctx, mapping, [1], atol=1e-05)}")

W matches: True
U matches: True
Wphy matches: True


iNeigh stores the nearest 32 neighbours for each template. Due to rounding errors this may not always match so we check to see how many nearest neighbours match. iList does exactly the same (repeated variable)

In [27]:
print(f"Number of nearest neighbours that match in iNeigh: \
{np.min(np.where((rez_split1.iNeigh-1)[mapping] != mapping[cp.asnumpy(ctx.intermediate.iNeigh_s.T)])[1])}/32")
print(f"Number of nearest neighbours that match in iList: \
{np.min(np.where((rez_split1.iList-1)[mapping] != mapping[cp.asnumpy(ctx.intermediate.iList.T)])[1])}/32")

Number of nearest neighbours that match in iNeigh: 21/32
Number of nearest neighbours that match in iList: 21/32


In [28]:
del ctx
del rez_split1

## Test Second Split

In [29]:
test_split2_path = setup_dir(test_path, 'test_split2')
rez_split1 = get_rez(rez_path / 'rez_split1.mat')

In [30]:
def save_rez_split1(rez, path):
   
    variables = ['ccbsort', 'dWU', 'igood', 'iList', 'iNeigh', 'iNeighPC', 'iorig', 'isplit', 'mu', 
                 'simScore', 'U', 'U_a', 'U_b', 'UA', 'W', 'W_a', 'W_b',
                'WA', 'Wrot', 'Wphy', 'wPCA', 'wTEMP']
    
    for variable in variables:
        _save(rez[variable], path, variable)
        
    _save(rez.ccb, path, 'ccb0')
    
    _save_largearray(rez.cProj, path, 'fW')
    _save_largearray(rez.cProjPC, path, 'fWPC')
    
    st3 = np.copy(rez.st3)
    st3[1,:] = rez.st3[1,:] - 1 # 0-index channels
    _save(st3, path, 'st3')
    _save(st3, path, 'st3_m')
    _save(st3, path, 'st3_s1')
    
    _save(rez.iNeigh, path, 'iNeigh_s')
    _save(rez.iNeighPC, path, 'iNeighPC_s')
    _save(rez.mu, path, 'mu_s')
    _save(rez.simScore, path, 'simScore_s')
    _save(rez.U, path, 'U_s')
    _save(rez.W, path, 'W_s')
    
    shutil.copyfile(path / 'fW.dat', path / 'proc.dat')

In [31]:
save_rez_split1(rez_split1, test_split2_path)
del rez_split1

In [32]:
probe = Bunch()
probe.NchanTOT = 385
# WARNING: indexing mismatch with MATLAB hence the -1
probe.chanMap = np.load(dir_path / 'chanMap.npy').squeeze().astype(np.int64) - 1
probe.xc = np.load(dir_path / 'xc.npy').squeeze()
probe.yc = np.load(dir_path / 'yc.npy').squeeze()
probe.kcoords = np.load(dir_path / 'kcoords.npy').squeeze()

In [33]:
ctx = run(dat_path, probe=probe, dir_path=test_split2_path.parent.parent, n_channels=385, dtype=np.int16, sample_rate=3e4, \
          stop_after='split_2')

23:26:02.808 [I] main:56              Loaded raw data with 385 channels, 3000000 samples.
23:26:03.043 [I] utils:334            Starting step split_2.
23:26:03.308 [I] postprocess:522      Found 0 splits, checked 0/376 clusters, nccg 0
23:26:07.380 [I] postprocess:522      Found 1 splits, checked 100/377 clusters, nccg 9
23:26:10.468 [I] postprocess:522      Found 1 splits, checked 200/377 clusters, nccg 10
23:26:14.338 [I] postprocess:522      Found 1 splits, checked 300/377 clusters, nccg 14
23:26:19.683 [I] postprocess:732      Finished splitting. Found 3 splits, checked 379/379 clusters, nccg 20
23:26:19.844 [I] utils:344            Step `split_2` took 16.80s.


In [34]:
rez_split2 = get_rez(rez_path / 'rez_split2.mat')

In [35]:
st3_matlab = np.copy(rez_split2.st3.T)
st3_matlab[:,1] = st3_matlab[:,1] - 1
st3_python = cp.asnumpy(ctx.intermediate.st3_s0)

In [36]:
ix = np.where(st3_matlab[:,1] - st3_python[:,1] != 0)[0]
bad_clusters = np.unique(np.concatenate((st3_python[:,1][ix], st3_matlab[:,1][ix])))
print(f'The following clusters were different: {bad_clusters}')

The following clusters were different: []


Some of these differences are just due to labelling, we can try and learn a mapping from the Python clusters to their corresponding Matlab clusters:

In [37]:
cluster_mapping = np.zeros((len(bad_clusters), 2), dtype = int)
cluster_mapping[:,0] = bad_clusters
cluster_mapping[:,1] = -1

for i in range(len(bad_clusters)):
    ix_p = np.where(st3_python[:,1] == bad_clusters[i])[0]
    for j in bad_clusters:
        ix_m = np.where(st3_matlab[:,1] == j)[0]
        if np.array_equal(ix_p, ix_m):
            cluster_mapping[i,1] = j
            break

mapping_found = False
if np.sum(cluster_mapping[:,1] == -1) > 0:
    print(f"No mapping found as the following Python clusters can't be matched:" \
        f"{cluster_mapping[:,0][np.where(cluster_mapping[:,1] == 0)[0]]}")
elif np.max(st3_matlab[:,1]) != np.max(st3_python[:,1]):
    print("Some Matlab clusters had no corresponding Python match")
else:
    mapping = np.arange(np.max(st3_python[:,1] + 1), dtype=int)
    for i in range(cluster_mapping.shape[0]):
        mapping[cluster_mapping[i,0]] = cluster_mapping[i,1]
    mapping_found = True
    print("Mapping was found")

Mapping was found


In [38]:
print(f"mu matches: {test('mu', rez_split2, ctx, mapping, [0], 'mu_s')}")
print(f"simScore matches: {test('simScore', rez_split2, ctx, mapping, [0,1], 'simScore_s')}")
print(f"isplit matches: {test('isplit', rez_split2, ctx, mapping, [0,1])}")

print(f"iNeighPC matches: {np.allclose(cp.asnumpy(ctx.intermediate.iNeighPC_s.T), rez_split2.iNeighPC - 1)}")

mu matches: True
simScore matches: True
isplit matches: True
iNeighPC matches: True


SVD components only match up to an arbitrary sign so we compare absolute values and also use a higher tolerance

In [39]:
print(f"W matches: {test_abs('W', rez_split2, ctx, mapping, [1], 'W_s', atol=1e-05)}")
print(f"U matches: {test_abs('U', rez_split2, ctx, mapping, [1], 'U_s', atol=1e-05)}")
print(f"Wphy matches: {test_abs('Wphy', rez_split2, ctx, mapping, [1], atol=1e-05)}")

W matches: True
U matches: True
Wphy matches: True


iNeigh stores the nearest 32 neighbours for each template. Due to rounding errors this may not always match so we check to see how many nearest neighbours match. iList does exactly the same (repeated variable)

In [40]:
print(f"Number of nearest neighbours that match in iNeigh: \
{np.min(np.where((rez_split2.iNeigh-1)[mapping] != mapping[cp.asnumpy(ctx.intermediate.iNeigh_s.T)])[1])}/32")
print(f"Number of nearest neighbours that match in iList: \
{np.min(np.where((rez_split2.iList-1)[mapping] != mapping[cp.asnumpy(ctx.intermediate.iList.T)])[1])}/32")

Number of nearest neighbours that match in iNeigh: 21/32
Number of nearest neighbours that match in iList: 21/32


In [41]:
del ctx
del rez_split2

## Test Cutoff

In [42]:
test_cutoff_path = setup_dir(test_path, 'test_cutoff')
rez_split2 = get_rez(rez_path / 'rez_split2.mat')

In [43]:
def save_rez_split2(rez, path):
   
    variables = ['ccbsort', 'dWU', 'igood', 'iList', 'iNeigh', 'iNeighPC', 'iorig', 'isplit', 'mu', 
                 'simScore', 'U', 'U_a', 'U_b', 'UA', 'W', 'W_a', 'W_b',
                'WA', 'Wrot', 'Wphy', 'wPCA', 'wTEMP']
    
    for variable in variables:
        _save(rez[variable], path, variable)
        
    _save(rez.ccb, path, 'ccb0')
    
    _save_largearray(rez.cProj, path, 'fW')
    _save_largearray(rez.cProjPC, path, 'fWPC')
    
    st3 = np.copy(rez.st3)
    st3[1,:] = rez.st3[1,:] - 1 # 0-index channels
    _save(st3, path, 'st3')
    _save(st3, path, 'st3_m')
    _save(st3, path, 'st3_s1')
    _save(st3, path, 'st3_s0')
    
    _save(rez.iNeigh, path, 'iNeigh_s')
    _save(rez.iNeighPC, path, 'iNeighPC_s')
    _save(rez.mu, path, 'mu_s')
    _save(rez.simScore, path, 'simScore_s')
    _save(rez.U, path, 'U_s')
    _save(rez.W, path, 'W_s')
    
    shutil.copyfile(path / 'fW.dat', path / 'proc.dat')

In [44]:
save_rez_split2(rez_split2, test_cutoff_path)
del rez_split2

In [45]:
probe = Bunch()
probe.NchanTOT = 385
# WARNING: indexing mismatch with MATLAB hence the -1
probe.chanMap = np.load(dir_path / 'chanMap.npy').squeeze().astype(np.int64) - 1
probe.xc = np.load(dir_path / 'xc.npy').squeeze()
probe.yc = np.load(dir_path / 'yc.npy').squeeze()
probe.kcoords = np.load(dir_path / 'kcoords.npy').squeeze()

In [46]:
ctx = run(dat_path, probe=probe, dir_path=test_cutoff_path.parent.parent, n_channels=385, dtype=np.int16, sample_rate=3e4, \
          stop_after='cutoff')

23:26:23.964 [I] main:56              Loaded raw data with 385 channels, 3000000 samples.
23:26:24.207 [I] utils:334            Starting step cutoff.
Setting cutoff: 100%|███████████████████████████████████████████████████████████████| 379/379 [00:02<00:00, 186.19it/s]
23:26:26.249 [I] utils:344            Step `cutoff` took 2.04s.


In [47]:
rez_cutoff = get_rez(rez_path / 'rez_cutoff.mat')

In [48]:
print(f"Ths matches: {np.allclose(rez_cutoff.Ths, ctx.intermediate.Ths)}")
print(f"good matches: {np.allclose(rez_cutoff.good, ctx.intermediate.good)}")
print(f"est_contam_rate matches: {np.allclose(rez_cutoff.est_contam_rate, ctx.intermediate.est_contam_rate)}")

st3_matlab = np.copy(rez_cutoff.st3.T)
st3_matlab[:,1] = st3_matlab[:,1] - 1
st3_python = cp.asnumpy(ctx.intermediate.st3_c)
print(f"st3 matches: {np.allclose(st3_python, st3_matlab)}")

Ths matches: True
good matches: True
est_contam_rate matches: True
st3 matches: True


In [49]:
del ctx
del rez_cutoff